In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# silence warnings
import warnings
warnings.filterwarnings("ignore")

# operating system
import os

# time calculation to track some processes
import time

# numeric and matrix operations
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
import datetime


# loading ploting libraries
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

# python core library for machine learning and data science
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.cluster import KMeans

RANDOM_STATE = 175

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD output/bbdd_clustering_v1.xlsx")

In [ ]:
df.info()

In [ ]:
del(df['Unnamed: 0'])
df.set_index("item", inplace = True)

In [ ]:
encoder = LabelEncoder()
df["category_id"] = encoder.fit_transform(df["category"])
df.drop("category", inplace = True, axis = 1)

In [ ]:
df['category_id'].value_counts()

In [ ]:
def report_df(df, verbose = True):
    '''
    Hace un report simple sobre el DataFrame suministrado.
    '''
    print(df.info(verbose = verbose))
    total_nulos = df.isnull().sum().sum()
    print()
    print(f"Tenemos un total de {total_nulos} nulos, correspondientes a ceros en ventas por año o por tienda")

In [ ]:
report_df(df)
df.head()

In [ ]:
df["2016_vs_2015"] = df.groupby("item")["2016_vs_2015"].apply(lambda series: series.backfill().ffill())
df["2015_vs_2014"] = df.groupby("item")["2015_vs_2014"].apply(lambda series: series.backfill().ffill())
df["2014_vs_2013"] = df.groupby("item")["2014_vs_2013"].apply(lambda series: series.backfill().ffill())
df["2012_vs_2011"] = df.groupby("item")["2012_vs_2011"].apply(lambda series: series.backfill().ffill())
df["weekend_sale_total %"] = df.groupby("item")["weekend_sale_total %"].apply(lambda series: series.backfill().ffill())
df["summer_sales_total %"] = df.groupby("item")["summer_sales_total %"].apply(lambda series: series.backfill().ffill())
df["christmas_sales_total %"] = df.groupby("item")["christmas_sales_total %"].apply(lambda series: series.backfill().ffill())
df["mean_discount"] = df.groupby("item")["mean_discount"].apply(lambda series: series.backfill().ffill())

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
report_df(df)

In [ ]:
df[df.duplicated(keep=False)]

In [ ]:
class ArrayToDataFrame(BaseEstimator, TransformerMixin):
    '''
    Clase que transforma un array en un DataFrame.
    Necesita como parámetros el nombre de las columnas y el índice.
    '''

    def __init__(self, columns, index = None):
        self.columns = columns
        self.index = index

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):

        if self.index != None:
            df = pd.DataFrame(X, columns = self.columns, index = self.index)

        else:
            df = pd.DataFrame(X, columns = self.columns)

        return df

In [ ]:
class OutlierFilter(BaseEstimator, TransformerMixin):
    '''
    Clase que filtra los outliers utilizando np.quantile()
    Los cuantiles a filtrar así como las columnas a filtrar son los parámetros de la clase.
    '''

    def __init__(self, q, col_to_filter):
        self.q = q
        self.col_to_filter = col_to_filter

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        '''
        El método considera outlier a aquel cliente que es outlier en todas las columnas que le pasas.
        Es decir: si tiene que filtrar importe y número de pedidos, sólo va a eliminar aquellos clientes
        que son outlier tanto en importe como número de pedidos. Si eres outlier en importe pero no en pedido
        no se te va a filtrar del dataset.
        '''

        # lista vacía
        criteria_list = []

        # agregamos a la lista los clientes que son outliers
        for col in self.col_to_filter:
            criteria = X[col] < np.quantile(X[col], q = self.q)
            criteria_list.append(criteria)

        # si hay más de 1 columna
        if len(self.col_to_filter) > 1:

            # creamos el criterio global: es decir outlier en todas las columnas
            global_criteria = criteria_list[0]

            for criteria in criteria_list[1:]:
                global_criteria = global_criteria & criteria

        else:
            global_criteria = criteria_list[0]

        # filtramos nuestra dataframe
        X = X[global_criteria]

        # guardamos el índice como parámetro de la clase porque en caso contrario lo perderíamos.
        self.index = X.index

        return X

In [ ]:
df_final = df.copy(deep = True)

In [ ]:
df_final.shape

In [ ]:
columns = list(df_final.columns)
index = list(df_final.index)

In [ ]:
df_final

In [ ]:
CALCULATE_ELBOW = True

In [ ]:
# separamos el pipeline del a loop, para no tener que volver a hacer los primeros pasos para cada k de la loop
pipe = Pipeline(steps = [
     ("Imputer", KNNImputer()),
     ("ArrayToDataFrame", ArrayToDataFrame(columns, index = index)),
     ("OutlierFilter", OutlierFilter(q = 0.99, col_to_filter = ['sales', 'mean_pvp', 'summer_sales_total %', 'weekend_sale_total %', 'christmas_sales_total %', 'exhibition_time', '2016_vs_2015', '2015_vs_2014','2014_vs_2013', 'mean_discount'])),
     ("StandardScaler", StandardScaler())
])

In [ ]:
 df_scaled_transformed_no_outliers = pipe.fit_transform(df_final)

In [ ]:
 if CALCULATE_ELBOW:
    st = time.time()

    sse = {}

    for k in range(2, 15):

        print(f"Fitting pipe with {k} clusters")

        clustering_model = KMeans(n_clusters = k)

        clustering_model.fit(df_scaled_transformed_no_outliers)

        sse[k] = clustering_model.inertia_

    et = time.time()
print("Elbow curve took {} seconds.".format(round((et - st)), 2))

In [ ]:
sse

In [ ]:
df_final.isnull().sum()

In [ ]:
if CALCULATE_ELBOW:
    fig = plt.figure(figsize = (16, 8))
    ax = fig.add_subplot()

    x_values = list(sse.keys())
    y_values = list(sse.values())

    ax.plot(x_values, y_values, label = "Inertia/dispersión de los clústers")
    fig.suptitle("Variación de la dispersión de los clústers en función de la k", fontsize = 16);

In [ ]:
 pipe = Pipeline(steps = [
     ("Imputer", KNNImputer()),
     ("ArrayToDataFrame", ArrayToDataFrame(columns, index = index)),
     ("OutlierFilter", OutlierFilter(q = 0.99, col_to_filter = ['sales', 'mean_pvp', 'summer_sales_total %', 'weekend_sale_total %', 'christmas_sales_total %', 'exhibition_time', '2016_vs_2015', '2015_vs_2014','2014_vs_2013', 'mean_discount'])),
     ("StandardScaler", StandardScaler()),
     ("Clustering", KMeans(n_clusters = 4))
 ])

In [ ]:
 df_final.shape

In [ ]:
pipe.fit(df_final)

In [ ]:
#  creamos un dataframe escalado con los pasos 1 - 2 y 4
X_processed = pipe[:2].transform(df_final)
X_scaled = pipe["StandardScaler"].transform(X_processed)
X_scaled.shape

In [ ]:
# hacemos el predict, en este caso tendremos para cada cliente su centroide/clúster.
labels = pipe["Clustering"].predict(X_scaled)

In [ ]:
pipe["Clustering"]

In [ ]:
 #le asignamos al DataFrame procesado los centroides.
 #SI LO HACEMOS AL ESCALADO LOS NÚMEROS PERDERAN SU SIGNIFICADO ESCALA Y SERÁN MÁS DÍFICILES DE INTERPRETAR.
X_processed["cluster"] = labels

In [ ]:
X_processed.shape

In [ ]:
# visualizamos nuestros grupos en base a las variables del modelo, para ver que tal han quedado.
selected_columns = ['sales', 'mean_pvp', 'summer_sales_total %', 'weekend_sale_total %', 'christmas_sales_total %', 'exhibition_time', '2016_vs_2015', '2015_vs_2014','2014_vs_2013', 'mean_discount']

# sns.pairplot(X_processed, vars = selected_columns, hue = 'cluster');

In [ ]:
ficha_df = pd.DataFrame()
ficha_df

In [ ]:
 for i, col in enumerate(['sales', 'mean_pvp', 'summer_sales_total %', 'weekend_sale_total %', 'christmas_sales_total %', 'exhibition_time', '2016_vs_2015', '2015_vs_2014','2014_vs_2013', 'mean_discount']):
     resumen_data = X_processed[["cluster", col]].groupby("cluster").describe().T[1:]
     ficha_df = ficha_df.append(resumen_data)

In [ ]:
 resumen_data

In [ ]:
# generamos nuestro multiindex
out_index = [
    "Ventas",
    "Precios",
    "Campañas",
    "Campañas",
    "Campañas",
    "Dias_en_venta",
    "Crecimiento",
    "Crecimiento",
    "Crecimiento",
    "Descuentos"
]

inner_index = [
    'sales',
    'mean_pvp',
    'summer_sales_total %',
    'weekend_sale_total %',
    'christmas_sales_total %',
    'exhibition_time',
    '2016_vs_2015',
    '2015_vs_2014',
    '2014_vs_2013',
    'mean_discount'
    ]

estadisticos = ["Media", "Desviación", "Mínimo", "Perc. 25", "Perc. 50", "Perc. 75", "Máximo"]

new_multi_index = []

for oi, ii, in zip(out_index, inner_index):
    for es in estadisticos:
        new_multi_index.append((oi, ii, es))

new_multi_index

In [ ]:
def generate_multiindex(list_of_tuples, names):
    return pd.MultiIndex.from_tuples(list_of_tuples, names = names)

In [ ]:
 names = ["Grupo Indicadores", "Indicador", "Estadístico"]
 index_ficha = generate_multiindex(new_multi_index, names)
 ficha_df.set_index(index_ficha, inplace = True)

In [ ]:
 tamaño_clusters = X_processed.groupby("cluster").size().to_frame().T
 tamaño_clusters.set_index(generate_multiindex([("General", "Clúster", "Tamaño")] , names), inplace = True)

In [ ]:
 ficha_df = tamaño_clusters.append(ficha_df)

In [ ]:
ficha_df

In [ ]:
ficha_df.style.background_gradient(cmap = 'Blues', axis = 1)

In [ ]:
ficha_df.to_excel("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD output/clusters_v1_7.xlsx")

In [ ]:
 X_processed['cluster']

In [ ]:
 X_cluster = X_processed['cluster']

In [ ]:
 cluster_0 = X_processed[X_processed["cluster"] == 0]

In [ ]:
 cluster_1 = X_processed[X_processed["cluster"] == 1]

In [ ]:
 cluster_2 = X_processed[X_processed["cluster"] == 2]

In [ ]:
cluster_3 = X_processed[X_processed["cluster"] == 3]

In [ ]:
X_cluster.to_excel("/content/drive/MyDrive/MASTER EN DATA SCIENCE/Capstone/data_dsmarket/GrupoRetail_1/Proceso Total/BBDD output/bd_grupos_1_7.xlsx")

**FIN**